In [74]:
from dotenv import load_dotenv
load_dotenv()

# .env 파일 강제 로드
load_dotenv(override=True)

True

In [8]:
import pandas as pd
preg_dur_df = pd.read_csv('C:\Workspaces\AI-Pharmacist\data_pregnancy\DUR_pregnant.csv',  encoding='cp949')

In [10]:
preg_dur_df.head()

,성분명,제품명,업체명,금기등급,상세정보
0,Chenodeoxycholic acid + Ursodeoxycholic acid,씨앤유캡슐(케노데속시콜산-우르소데속시콜산삼수화물마그네슘염)_(0.25g/1캡슐),명문제약(주),1,"임부 투여금기, 동물실험에서 간독성의 증거 나타남"
1,acamprosate,명인아캄프로세이트정333mg(아캄프로세이트칼슘)_(0.333g/1정),명인제약(주),2,임부에 대한 안전성 미확립.
2,acamprosate,환인아캄프로세이트정(아캄프로세이트칼슘)_(0.333g/1정),환인제약(주),2,임부에 대한 안전성 미확립.
3,acarbose,글루코바이정100밀리그람(아카보즈)_(0.1g/1정),(주)한독,2,임부에 대한 안전성 미확립.
4,acarbose,글루코바이정50밀리그람(아카보즈)_(50mg/1정),(주)한독,2,임부에 대한 안전성 미확립.


In [75]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('C:\Workspaces\AI-Pharmacist\data_pregnancy\DUR_pregnant.csv', encoding='cp949')
preg_dur_document = loader.load()
print(type(preg_dur_document))
print(len(preg_dur_document))
print(preg_dur_document[0])
print(type(preg_dur_document[0]))

<class 'list'>
19014
page_content='성분명: Chenodeoxycholic acid + Ursodeoxycholic acid
제품명: 씨앤유캡슐(케노데속시콜산-우르소데속시콜산삼수화물마그네슘염)_(0.25g/1캡슐)
업체명: 명문제약(주)
금기등급: 1
상세정보: 임부 투여금기, 동물실험에서 간독성의 증거 나타남' metadata={'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR_pregnant.csv', 'row': 0}
<class 'langchain_core.documents.base.Document'>


In [76]:
import os
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

print(PINECONE_INDEX_NAME)
print(PINECONE_NAMESPACE)
print(PINECONE_API_KEY)

dur-pregnancy
dur-pregnancy-ns
pcsk_5TkRik_EzGTnXtpN1c26rjgbrhySAUqgAAHhR9QEmje4vwZkMoEKiADwUKrPjkEAQwfsTs


In [13]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [77]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os

PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 문서 배치 크기 설정 (예: 1000개씩 나눔)
batch_size = 1000
total_documents = len(preg_dur_document)
# # 문서에 대한 임베딩 생성
# document_embeddings = embeddings.embed_documents([doc.page_content for doc in preg_dur_document])

# 문서를 배치로 나누어 처리
for i in range(0, total_documents, batch_size):
    batch = preg_dur_document[i:i+batch_size]
    # embeddings_batch = document_embeddings[i:i+batch_size]  # 배치에 맞는 임베딩
    
    # Pinecone에 배치 업로드
    vector_db = PineconeVectorStore.from_documents(
        batch,
        embedding=embeddings,
        index_name=PINECONE_INDEX_NAME,
        namespace=PINECONE_NAMESPACE,
        pinecone_api_key=PINECONE_API_KEY
    )

    # 배치 업로드 완료 메시지
    print(f"Batch {i // batch_size + 1} uploaded, {min(i + batch_size, total_documents)} documents processed.")

Batch 1 uploaded, 1000 documents processed.
Batch 2 uploaded, 2000 documents processed.
Batch 3 uploaded, 3000 documents processed.
Batch 4 uploaded, 4000 documents processed.
Batch 5 uploaded, 5000 documents processed.
Batch 6 uploaded, 6000 documents processed.
Batch 7 uploaded, 7000 documents processed.
Batch 8 uploaded, 8000 documents processed.
Batch 9 uploaded, 9000 documents processed.
Batch 10 uploaded, 10000 documents processed.
Batch 11 uploaded, 11000 documents processed.
Batch 12 uploaded, 12000 documents processed.
Batch 13 uploaded, 13000 documents processed.
Batch 14 uploaded, 14000 documents processed.
Batch 15 uploaded, 15000 documents processed.
Batch 16 uploaded, 16000 documents processed.
Batch 17 uploaded, 17000 documents processed.
Batch 18 uploaded, 18000 documents processed.
Batch 19 uploaded, 19000 documents processed.
Batch 20 uploaded, 19014 documents processed.


In [71]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os

# Pinecone 초기화
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),        # 환경 변수에서 API 키 불러오기
    environment=os.getenv("PINECONE_ENVIRONMENT")  # 환경 변수에서 환경 설정 불러오기
)

# 인덱스 객체 생성
index = pinecone.Index(os.getenv("PINECONE_INDEX_NAME"))

# 네임스페이스 내 모든 데이터 삭제
index.delete(delete_all=True, namespace=os.getenv("PINECONE_NAMESPACE"))

print("네임스페이스 내 데이터가 성공적으로 삭제되었습니다.")

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [40]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ('system', """
Persona: 당신은 전문적인 약사입니다. 약에 대해 복용 방법, 주의 사항, 금기 여부를 알고 있으며 일반인을 대상으로 설명이 가능합니다.

Role: 임산부에게 의약품의 복용 가능 여부 및 금기 등급, 금기 등급의 의미를 정확히 이해할 수 있도록 돕습니다. context를 기반으로 약품에 대해 아무것도 모르는 임산부가 이해할 수 있도록 주의 사항을 도출해서 5줄 이내로 알려줍니다.

Examples: 만약 해당 성분이 임산부에게 해로우면, 해당 의약품이 어떤 성분으로 인해 어느 금기 등급에 속하며 어떤 위험성이 있는지 알려준다. 만약 해당 성분이 임산부에게 해롭지 않으면, DUR 임부금기에 해당되지 않으므로 복용해도 된다고 알려준다.
"""),
    ('user', '''
    query 약품을 임산부가 먹어도 되는지 context만 참조해서 알려줘.
    
    question: {query}  
    context: {context}
    ''')
])
prompt

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nPersona: 당신은 전문적인 약사입니다. 약에 대해 복용 방법, 주의 사항, 금기 여부를 알고 있으며 일반인을 대상으로 설명이 가능합니다.\n\nRole: 임산부에게 의약품의 복용 가능 여부 및 금기 등급, 금기 등급의 의미를 정확히 이해할 수 있도록 돕습니다. context를 기반으로 약품에 대해 아무것도 모르는 임산부가 이해할 수 있도록 주의 사항을 도출해서 5줄 이내로 알려줍니다.\n\nExamples: 만약 해당 성분이 임산부에게 해로우면, 해당 의약품이 어떤 성분으로 인해 어느 금기 등급에 속하며 어떤 위험성이 있는지 알려준다. 만약 해당 성분이 임산부에게 해롭지 않으면, DUR 임부금기에 해당되지 않으므로 복용해도 된다고 알려준다.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template='\n    query 약품을 임산부가 먹어도 되는지 context만 참조해서 알려줘.\n    \n    question: {query}  \n    context: {context}\n    '), additional_kwargs={})])

In [41]:
query = "에이서캡슐(아세클로페낙)_(0.1g/1캡슐)"
retriever = vector_db.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 5}
)
retrievals = retriever.batch([query])
retrievals

[[Document(id='70229fc7-96c4-43fd-a87f-117bc6478860', metadata={'row': 16294.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR_pregnant.csv'}, page_content='성분명: ginkgo biloba leaf ext.\n제품명: 써큐로드연질캡슐120mg(은행엽건조엑스)_(0.12g/1캡슐)\n업체명: (주)동구바이오제약\n금기등급: 2\n상세정보: (정제, 액제, 캡슐제) 임신 중 투여에 대한 충분한 사용경험 적음 (주사제) -'),
  Document(id='24b867a7-929b-4642-93da-c148e075c349', metadata={'row': 1763.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR_pregnant.csv'}, page_content='성분명: Diacerein\n제품명: 아세렌캡슐(디아세레인)_(50mg/1캡슐)\n업체명: 대화제약(주)\n금기등급: 2\n상세정보: 동물실험에서 태자 골형성 지연 보고'),
  Document(id='7066ca9f-d47f-4a59-a425-d8b53f5e34ce', metadata={'row': 18984.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR_pregnant.csv'}, page_content='성분명: pseudoephedrine\n제품명: 쏙콜에이연질캡슐_(1캡슐)\n업체명: (주)에이프로젠바이오로직스\n금기등급: 2\n상세정보: 임부에 대한 안전성 미확립.'),
  Document(id='93881303-1e56-4b69-9248-4beed2a761f1', metadata={'row': 5.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR

In [42]:
prompt.invoke({'query': query, 'context': retriever})

ChatPromptValue(messages=[SystemMessage(content='\nPersona: 당신은 전문적인 약사입니다. 약에 대해 복용 방법, 주의 사항, 금기 여부를 알고 있으며 일반인을 대상으로 설명이 가능합니다.\n\nRole: 임산부에게 의약품의 복용 가능 여부 및 금기 등급, 금기 등급의 의미를 정확히 이해할 수 있도록 돕습니다. context를 기반으로 약품에 대해 아무것도 모르는 임산부가 이해할 수 있도록 주의 사항을 도출해서 5줄 이내로 알려줍니다.\n\nExamples: 만약 해당 성분이 임산부에게 해로우면, 해당 의약품이 어떤 성분으로 인해 어느 금기 등급에 속하며 어떤 위험성이 있는지 알려준다. 만약 해당 성분이 임산부에게 해롭지 않으면, DUR 임부금기에 해당되지 않으므로 복용해도 된다고 알려준다.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content="\n    query 약품을 임산부가 먹어도 되는지 context만 참조해서 알려줘.\n    \n    question: 에이서캡슐(아세클로페낙)_(0.1g/1캡슐)  \n    context: tags=['PineconeVectorStore', 'OpenAIEmbeddings'] vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000027FC6DF8590> search_kwargs={'k': 5}\n    ", additional_kwargs={}, response_metadata={})])

In [34]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o',
    temperature=0.5
)

In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# RunnablePassThrough: invoke()에 전달된 인자를 받아서 그대로 출력
chain = {'query': RunnablePassthrough(), 'context': retriever} | prompt | model | StrOutputParser() 

chain.invoke(query)

'\n에이서캡슐(아세클로페낙)은 임산부에게 금기 등급 2에 해당합니다. 이는 임부에 대한 안전성이 확립되지 않았으며, 임신 말기에 투여 시 태아의 동맥관 조기 폐쇄 가능성이 있습니다. 또한, 비스테로이드성 소염진통제는 난산 발생 빈도 증가, 분만 지연, 태아 생존율 감소를 일으킬 수 있습니다. 임신 약 20주 이후 사용 시 태아의 신기능 이상을 초래하여 양수 과소증 및 신생아 신장애 발생 가능성이 보고되었습니다. 따라서 임산부는 이 약의 복용을 피하는 것이 좋습니다.'

In [44]:
chain.invoke('덱스트로메토펜')

'\n덱스트로메토펜(Dextromethorphan)에 대한 정보는 제공된 context에 포함되어 있지 않습니다. 그러나 일반적으로 덱스트로메토펜은 임신 중에도 비교적 안전한 것으로 간주되며, DUR 임부금기 등급에 해당되지 않습니다. 하지만, 모든 약물 복용은 개인의 건강 상태와 임신 주기에 따라 다를 수 있으므로, 복용 전에 반드시 담당 의사나 약사와 상담하는 것이 중요합니다.'

In [45]:
chain.invoke('아세렌캡슐(디아세레인')

'\n아세렌캡슐(디아세레인)의 주성분인 디아세레인은 임산부에게 금기등급 2에 속합니다. 이는 동물실험에서 태아의 골형성 지연이 보고되었음을 의미합니다. 임신 중 이 약을 복용하면 태아에게 해로울 수 있으므로, 복용을 피하는 것이 좋습니다. 의사와 상의하여 대체 약물을 고려하는 것이 안전합니다.'

In [46]:
chain.invoke('타이레놀')

'\n타이레놀 자체는 일반적으로 임산부에게 안전한 것으로 알려져 있지만, 타이레놀콜드-에스정에는 Pseudoephedrine이라는 성분이 포함되어 있습니다. 이 성분의 경우 임부에 대한 안전성이 확립되지 않았으며, 금기등급 2로 분류되어 있습니다. 이는 임산부에게 잠재적인 위험이 있을 수 있음을 의미합니다. 따라서, 타이레놀콜드-에스정은 임산부가 복용하기에 적합하지 않을 수 있으니, 타이레놀을 복용하려면 반드시 주치의와 상담하시기 바랍니다.'

In [47]:
chain.invoke('게보린정')

'\n게보린정은 acetaminophen, caffeine anhydrous, isopropylantipyrine 성분을 포함하고 있으며, 임산부에게 금기등급 2에 해당합니다. 금기등급 2는 임부에 대한 안전성이 확립되지 않았거나, 잠재적인 위험성이 있을 수 있음을 의미합니다. 따라서 임산부는 이 약을 복용하기 전에 반드시 의료 전문가와 상의하는 것이 중요합니다.'

In [65]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [78]:
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

dur_vector_db = PineconeVectorStore(
    embedding=embeddings,
    index_name=PINECONE_INDEX_NAME,
    namespace=PINECONE_NAMESPACE,
    pinecone_api_key=PINECONE_API_KEY
)

In [79]:
dur_retriever = dur_vector_db.as_retriever(    
    search_type='similarity',
    search_kwargs={'k': 5})

In [80]:
retrievals = dur_retriever.batch(['에이서캡슐(아세클로페낙) 임산부가 먹어도될까?'])
retrievals

[[Document(id='df8b97a9-a8df-4113-805a-8ea578f3c641', metadata={'row': 6454.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR_pregnant.csv'}, page_content='성분명: Roxatidine\n제품명: 록사겐캡슐(염산록사티딘아세테이트)_(75mg/1캡슐)\n업체명: 신풍제약(주)\n금기등급: 2\n상세정보: 임부에 대한 안전성 미확립'),
  Document(id='0d2ed571-1743-41dd-bdcb-b2d2982d88e3', metadata={'row': 3773.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR_pregnant.csv'}, page_content='성분명: itraconazole\n제품명: 티나덤캡슐(이트라코나졸)_(0.1g/1캡슐)\n업체명: 동광제약(주)\n금기등급: 2\n상세정보: 임부에 대한 안전성 미확립.동물실험에서 고용량 투여시 태자 기형발생 증가, 배자독성 유발됨.'),
  Document(id='695d3575-4b63-43fe-899c-29d14e4b9f04', metadata={'row': 5.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR_pregnant.csv'}, page_content='성분명: aceclofenac\n제품명: 에이서캡슐(아세클로페낙)_(0.1g/1캡슐)\n업체명: 경동제약(주)\n금기등급: 2\n상세정보: 임부에 대한 안전성 미확립.임신 말기에 투여시 태아의 동맥관조기폐쇄 가능성.동물실험에서 비스테로이드성 소염진통제는 난산발생빈도 증가, 분만지연, 태아 생존율 감소 보고.임신 약 20주 이후 비스테로이드성 소염제의 사용은 태아의 신기능 이상을 일으켜 양수 과소증 유발 가능 및 경우에 따라 신생아 신장애 발생 가능'),


In [85]:
result = dur_vector_db.similarity_search(    
    query='타이레놀' ,
    k=5,
    namespace=PINECONE_NAMESPACE)
print(result)

[Document(id='6b771f35-b37b-4467-991c-8b53f8ffacb0', metadata={'row': 11577.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR_pregnant.csv'}, page_content='성분명: tigecycline\n제품명: 타이가실주(타이제사이클린)_(50mg/1병)\n업체명: 한국화이자제약(주)\n금기등급: 2\n상세정보: 테트라사이클린계 항생제와 같이 이 약은 임신 2-3분기 동안 투여할 때 영구적인 젖니 변색 및 가역적인 뼈 성장 억제를 유발할 수 있음. 기관 형성 기간 동안 임신한 랫트 및 토끼에게 사람에서 권장된 임상용량의 각각 5배 및 1배에 해당하는 노출 시 태자의 체중 감소 및 부골격이상(골화의 지연) 발생률 증가와 연관있음.'), Document(id='042774ad-eb50-4f4f-a20f-7c39671a901d', metadata={'row': 18475.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR_pregnant.csv'}, page_content='성분명: vitamin A (retinol) 함유 복합제\n제품명: 비타에이트큐정_(1정)\n업체명: (주)시어스제약\n금기등급: 1\n상세정보: vitamin A (retinol): 임신초기 3개월간 고용량 투여시 최기형 가능성.(단, vitamin A (retinol)로서 5000 I.U/1일 이상)'), Document(id='b9738e37-ec7c-496e-a493-6b1560888c1d', metadata={'row': 16798.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR_pregnant.csv'}, page_content='성분명: Pseudoephedrine\n제품명: 타이레놀콜드-에스정_(1정)\n업체명: (주

In [83]:
print(result)

[Document(id='695d3575-4b63-43fe-899c-29d14e4b9f04', metadata={'row': 5.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\data_pregnancy\\DUR_pregnant.csv'}, page_content='성분명: aceclofenac\n제품명: 에이서캡슐(아세클로페낙)_(0.1g/1캡슐)\n업체명: 경동제약(주)\n금기등급: 2\n상세정보: 임부에 대한 안전성 미확립.임신 말기에 투여시 태아의 동맥관조기폐쇄 가능성.동물실험에서 비스테로이드성 소염진통제는 난산발생빈도 증가, 분만지연, 태아 생존율 감소 보고.임신 약 20주 이후 비스테로이드성 소염제의 사용은 태아의 신기능 이상을 일으켜 양수 과소증 유발 가능 및 경우에 따라 신생아 신장애 발생 가능')]


In [ ]:

prompt.invoke({'query': query, 'context': retriever})

In [62]:
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

In [54]:
output_parser = StrOutputParser()

In [59]:
rag_chain = ({'context': dur_retriever, 'question': RunnablePassthrough()}| prompt | model | output_parser)

In [60]:
rag_chain.invoke('에이서캡슐(아세클로페낙) 임산부가 먹어도될까?')

'해당 문서에서는 에이서캡슐(아세클로페낙)에 대한 정보가 제공되지 않았습니다. 따라서 임산부가 이 약을 복용해도 되는지에 대한 정보는 알 수 없습니다. 의사나 약사와 상담하는 것이 좋습니다.'

In [ ]:
chain = {'query': RunnablePassthrough(), 'context': retriever} | prompt | model | StrOutputParser() 